# Macroprudential Policy and the Housing Market
This notebook solves and simulates the Heterogenous Agent Housing Market (HAH) model and produces the output contained in my Master's thesis. 

**Dependencies:**

The code structure builds upon the framework developed by Jeppe Druedahl & Co. in the [NumEconCopenhagen Project](https://github.com/NumEconCopenhagen)

Packages required for running the notebooks are:
- [ConSav](https://pypi.org/project/ConSav/)
- [EconModel](https://pypi.org/project/EconModel/)
- [matplotlib](https://pypi.org/project/matplotlib/)
- [numpy](https://pypi.org/project/numpy/)
- [numba](https://pypi.org/project/numba/)

In [14]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import numba as nb
nb.set_num_threads(4)

from HAHModel import HAHModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Baseline model

In [15]:
model_baseline = HAHModelClass(
    name='baseline',
    par={
        'do_print':True
        })

#model_baseline.precompile_numba() # solve with very coarse grids
#model_baseline.solve() 
#model_baseline.simulate()

### Test solving the household problem for a single period

In [64]:
import trans
import HHproblems as hhp

t = par.T-1

with jit(model_baseline) as model: 
    sol = model.sol
    par = model.par
    #hhp.last_period_v_bar_q(t,sol,par)
    #print('bequest is done')
    #hhp.solve_stay(t,sol,par)
    #print('stay is done')
    #hhp.solve_ref(t,sol,par)
    #print('refinance is done')
    #hhp.solve_buy(t,sol,par)
    #print('buy is done')
    #hhp.solve_rent(t,sol,par)
    #print('rent is done')
    t = par.T-2
    hhp.postdecision_compute_v_bar_q.py_func(t,sol,par)
    print('post decision is done')
    #assert np.all((sol.c_stay[t] >= 0) & (np.isnan(sol.c_stay[t]) == False))
    #assert np.all((sol.inv_v_bar[t] >= 0) & (np.isnan(sol.inv_v_bar[t]) == False))



[0 0 0 0 0 0 0 0 0 0 0]
prep_own is [0 0 0 0 0 0 0 0 0 0 0 0]
par.grid_m is [ 0.          0.12769237  0.32060611  0.61205431  1.05236535  1.71757384
  2.72255053  4.24083866  6.53462204 10.        ]
grid_d_prime is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
par.grid_w is [0.39126563 0.52373855 0.70106355 0.93842643 1.25615454 1.68145758
 2.25075777]
solution array is [[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 

Traceback (most recent call last):
  File "/var/folders/23/m5wgdyb92cs9fl4fls0nt3xw0000gn/T/ipykernel_26522/1081918203.py", line 20, in <cell line: 6>
    hhp.postdecision_compute_v_bar_q.py_func(t,sol,par)
  File "/Users/Christian/Dropbox/KU - Polit/KA/Thesis/MScThesis-2022/HHproblems.py", line 220, in postdecision_compute_v_bar_q
    linear_interp.interp_3d_only_last_vec_mon(prep_own,grid_d_prime,par.grid_w,par.grid_m,
ZeroDivisionError: division by zero


ZeroDivisionError: division by zero

### Testing utility functions, tax scheme and mortgage payment

In [16]:
import mt
import utility
from EconModel import jit

with jit(model_baseline) as model_jit:
    par = model_jit.par
    
    # mortgage schedule
    annuity,pr_rem,interest,pr_pmt = mt.mpmt(10**6,1,20,0,par)

    # income and property tax
    tax_h = mt.property_tax(1.0,2.0,par)
    ytilde = mt.income_aftertax(2.2,0.8,3.5,0,par)

    # utility
    u_stay = utility.func(1,2,0,0,5,par)
    marg_u_stay = utility.marg_func(2,par,par.T-1)
    u_move = utility.func(1,2,1,0,5,par)
    u_rent = utility.func(1,2,1,1,5,par)

# print output
print(f'the full annuity payment is {round(annuity,2)}')
print(f'the remaining principal is {round(pr_rem,2)}')
print(f'the interest payment is {round(interest,2)}')
print(f'the principal payment is {round(pr_pmt,2)}')
print('--------------------------------------------')
print(f'the property tax is {round(tax_h,2)}')
print(f'the after tax income is {round(ytilde,2)}')
print('--------------------------------------------')
print(f'utility for a stayer is {round(u_stay,2)}')
print(f'marginal stay utility is {round(marg_u_stay,2)}')
print(f'utility for a mover is {round(u_move,2)}')
print(f'utility for a renter is {round(u_rent,2)}')


the full annuity payment is 67215.71
the remaining principal is 962784.29
the interest payment is 30000.0
the principal payment is 37215.71
--------------------------------------------
the property tax is 0.02
the after tax income is 1.57
--------------------------------------------
utility for a stayer is -3.17
marginal stay utility is 0.18
utility for a mover is -3.51
utility for a renter is -3.55


### Testing transition rules

In [17]:
import trans
# cash on hand

# da periods
Tda_next = trans.Tda_plus_func(10)
print(Tda_next)

with jit(model_baseline) as model_jit:
    par = model_jit.par
    
    # mortgage balance
    d_plus = trans.d_plus_func(q=1,h=2,d=1.5,w=0.8,move=1,ref=0,t=5,Td=20,Tda=0,par=par)

print(f'next period mortgage balance is {d_plus}') 

# income states

9
next period mortgage balance is 1.6480000000000001


In [48]:
sol = model_baseline.sol
par = model_baseline.par
sol.inv_v_stay[par.T-1,:,0,:,0,0,0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.04696467, 0.04696467, 0.04696467, 0.04696467, 0.04696467,
        0.04696467, 0.04696467, 0.04696467, 0.04696467, 0.04696467],
       [0.05616226, 0.05616226, 0.05616226, 0.05616226, 0.05616226,
        0.05616226, 0.05616226, 0.05616226, 0.05616226, 0.05616226],
       [0.05985707, 0.05985707, 0.05985707, 0.05985707, 0.05985707,
        0.05985707, 0.05985707, 0.05985707, 0.05985707, 0.05985707],
       [0.06257514, 0.06257514, 0.06257514, 0.06257514, 0.06257514,
        0.06257514, 0.06257514, 0.06257514, 0.06257514, 0.06257514],
       [0.06657101, 0.06657101, 0.06657101, 0.06657101, 0.06657101,
        0.06657101, 0.06657101, 0.06657101, 0.06657101, 0.06657101],
       [0.07243067, 0.07243067, 0.07243067, 0.07243067, 0.07243067,
        0.07243067, 0.07243067, 0.07243067, 0.07243067, 0.07243067],
       [0.08090088, 0.08090088, 0.0809008

### Sanity checks

In [18]:
par = model_baseline.par
h = par.grid_h[1]
w = par.w_grid[-1]
grid_d_prime = par.grid_d_prime  

d_prime_high = np.fmin(par.omega_ltv*par.q*h,par.omega_dti*w) 
i_dp_max = 0
while grid_d_prime[i_dp_max] < d_prime_high:
    i_dp_max += 1
    print(i_dp_max)
if i_dp_max < len(grid_d_prime):                      
    grid_d_prime = np.append(grid_d_prime[0:i_dp_max],d_prime_high)

print(f'length of capped mortgage grid is {len(grid_d_prime)}')
print(f'length of original mortgage grid is {len(par.grid_d_prime)}')


1
2
3
4
5
6
length of capped mortgage grid is 7
length of original mortgage grid is 10


In [10]:
#1/utility.marg_func(model_baseline.sol.c_stay[par.T-1,3,0,0,0,0,6],par,par.T-1)

#model_baseline.sol.c_stay[t].shape

np.nan*np.zeros(1)

array([nan])

In [17]:
model_baseline.par.w_grid 
model_baseline.par.w_trans.round(2)

array([[0.89, 0.11, 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.02, 0.89, 0.09, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.04, 0.89, 0.07, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.05, 0.89, 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.07, 0.89, 0.04, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.09, 0.89, 0.02],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.11, 0.89]])